<a href="https://colab.research.google.com/github/pthmhatre/PRODIGY_GA_01/blob/main/TASK01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch transformers datasets


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import torch

with open('/content/Model/Story.txt', 'r') as file:
    data = file.read()

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token


tokenized_inputs = tokenizer(data, return_tensors='pt', truncation=True, padding=True, max_length=512)

labels = tokenized_inputs['input_ids'].clone()
labels[labels == tokenizer.pad_token_id] = -100

dataset = Dataset.from_dict({
    'input_ids': tokenized_inputs['input_ids'],
    'attention_mask': tokenized_inputs['attention_mask'],
    'labels': labels
})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments


model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=5, training_loss=1.1219844818115234, metrics={'train_runtime': 76.936, 'train_samples_per_second': 0.065, 'train_steps_per_second': 0.065, 'total_flos': 1306460160000.0, 'train_loss': 1.1219844818115234, 'epoch': 5.0})

In [ ]:
model.save_pretrained('/content/Model')
tokenizer.save_pretrained('/content/Model')

('/content/Model/tokenizer_config.json',
 '/content/Model/special_tokens_map.json',
 '/content/Model/vocab.json',
 '/content/Model/merges.txt',
 '/content/Model/added_tokens.json')

In [ ]:
prompt = "At three o'clock precisely I was at Baker Street"
inputs = tokenizer(prompt, return_tensors='pt')
output = model.generate(**inputs, max_length=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


At three o'clock precisely I was at Baker Street, and I was in the middle of the street, and I was in the middle of the street, and I was in the middle of the street, and I was in the middle of the street, and I was in the middle of the street, and I was in the middle of the middle of the street, and I was in the middle of the street, and I was in the middle of the street, and I was in the
